In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('----------- Float tensor set --------------')
else:
    print('---------------------- No CUDA -----------------')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "BAAI/bge-large-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
# text = 'fantasy'

# inputs = tokenizer(text, return_tensors='pt')

# token_ids = inputs['input_ids'][0]
# tokens = tokenizer.convert_ids_to_tokens(token_ids)
# print(tokens)

# with torch.no_grad():
#     outputs = model(**inputs, output_hidden_states=True)

# last_hidden_state = outputs.last_hidden_state
# last_fantasy_layer = outputs.last_hidden_state[0][1].clone() # fantasy token embedding
# pooled_output = outputs.pooler_output[0]
# print(last_hidden_state.shape)
# print(pooled_output.shape)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


----------- Float tensor set --------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [17]:
valid_token_ids = []
for id in range(tokenizer.vocab_size):
    token = tokenizer.convert_ids_to_tokens(id)
    # Filter for actual words - you may need to adjust these conditions for BGE
    if (
        not token.startswith('[') and  # Skip special tokens
        not token.startswith('##') and  # Skip subword pieces
        not any(c in token for c in '〜་『』«»‰―⟩（') and  # Skip special characters
        # len(token) > 1 and  # Skip single characters
        token.isascii()  # Only keep ASCII tokens
    ):
        valid_token_ids.append(id)
valid_embeddings = model.embeddings.word_embeddings(torch.tensor(valid_token_ids))

In [10]:
print(tokenizer('The Wheel of Time turns, and Ages come and pass, leaving memories that become legend', return_tensors='pt'))

{'input_ids': tensor([[ 101, 1996, 5217, 1997, 2051, 4332, 1010, 1998, 5535, 2272, 1998, 3413,
         1010, 2975, 5758, 2008, 2468, 5722,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

def layerOutput(layerNum, hiddenInput):
    layer = model.encoder.layer[layerNum]
    attention_output = layer.attention(hiddenInput)[0]  # Returns tuple, we want first element
    intermediate_output = layer.intermediate(attention_output)
    layer_output = layer.output(intermediate_output, attention_output)
    return layer_output

def startToEndLayerOutput(start, end, hiddenInput):
    layer_output = hiddenInput
    for layerNum in range(start, end+1):
        layer_output = layerOutput(layerNum, layer_output)

    return layer_output

def getInputToPool(output_last_state, learnedHidden):
    learnedHiddenLayer = output_last_state.clone()
    learnedHiddenLayer[:, 0] = learned_hidden
    return learnedHiddenLayer

def getFullHiddenLayer(hiddenVals, cls, sep):
    return torch.cat([
        cls.unsqueeze(0).unsqueeze(0),
        hiddenVals.unsqueeze(0),
        sep.unsqueeze(0).unsqueeze(0)
    ], dim=1)

def getHiddenLayerInput(startingLayerNum, hiddenVals, cls, sep):
    if startingLayerNum == 0:
        return getFullHiddenLayer(hiddenVals, cls, sep)
    else:
        return hiddenVals

def project_to_valid_embeddings(continuous_embeddings, temperature=1.0):
    # Normalize both embeddings for cosine similarity
    continuous_norm = F.normalize(continuous_embeddings, p=2, dim=1)
    valid_norm = F.normalize(valid_embeddings, p=2, dim=1)
    similarities = torch.matmul(continuous_embeddings, valid_embeddings.T)
    # Gumbel-softmax allows gradients to flow while approximating discrete sampling
    soft_indices = F.gumbel_softmax(similarities / temperature, dim=1)
    return torch.matmul(soft_indices, valid_embeddings)

startLayerNum = 0
targetLayerNum = 23
targetText = 'The Wheel of Time turns, and Ages come and pass, leaving memories that become legend'
# Get the magic embedding and fantasy hidden states
with torch.no_grad():
    # Get magic's pooled embedding
    magic_output = model(**tokenizer(targetText, return_tensors='pt'), output_hidden_states=True)
    # magic_input = magic_output.hidden_states[targetLayerNum-1]
    magic_embedding = magic_output.pooler_output

    # Get fantasy's last hidden states
    # tokenInputs = tokenizer('keypad', return_tensors='pt')
    numTokensInput = (2) + 17 # start and end token + how many actual tokens
    fantasy_output = model(
        # input_ids=torch.tensor([[101] + [4633] * (numTokensInput - 2) + [102]]),
        input_ids=torch.tensor([[ 101, 5217, 1997, 2051, 4332, 1010, 1998, 5535, 2272, 1998, 3413,
         1010, 2975, 5758, 2008, 2468, 5722,  102]]),
        attention_mask=torch.tensor([[1] * numTokensInput]),
        output_hidden_states=True
    )
    # fantasy_output = model(**tokenizer('fantasy', return_tensors='pt'), output_hidden_states=True)
    cls_embedding = fantasy_output.hidden_states[0][0][0].clone()
    sep_embedding = fantasy_output.hidden_states[0][0][-1].clone()
    fantasy_hidden = fantasy_output.hidden_states[startLayerNum][0]

    if startLayerNum == 0:
        fantasy_hidden = fantasy_hidden[1:-1] # optimize only the actual tokens not cls and sep

# Create learnable hidden states initialized with fantasy's hidden states
learned_hidden = nn.Parameter(fantasy_hidden.clone(), requires_grad=True)

# Setup optimizer
optimizer = optim.Adam([learned_hidden], lr=0.1)

# Training loop
n_steps = 2500
for step in range(n_steps+1):
    layer_output = startToEndLayerOutput(startLayerNum, targetLayerNum, getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding))
    layer_output = model.pooler(layer_output)

    # Compute MSE loss between pooled result and magic embedding
    mseLoss = nn.MSELoss()(layer_output, magic_embedding)
    cosineLoss = 1 - nn.CosineSimilarity(dim=1)(layer_output, magic_embedding)

    # Backward pass and optimization
    optimizer.zero_grad()
    cosineLoss.backward()
    optimizer.step()

    temperature = max(1.0 * math.exp(-step/500), 0.1)  # Exponential decay from 1.0 to 0.1
    # snap learned hidden dims to nearest token
    learned_hidden.data = project_to_valid_embeddings(learned_hidden.data, temperature)

    if cosineLoss.item() < 1e-12:
        print("Reached zero error!")
        break

    # Print progress every 100 steps
    if (step + 1) % 100 == 0:
        print(f'Step {step + 1}/{n_steps}, MSE: {mseLoss.item()}, Cosine: {cosineLoss.item()}')

Step 100/2500, MSE: 0.18673980236053467, Cosine: 0.1636943817138672
Step 200/2500, MSE: 0.1867569386959076, Cosine: 0.16365522146224976
Step 300/2500, MSE: 0.18700188398361206, Cosine: 0.1636427640914917
Step 400/2500, MSE: 0.18720334768295288, Cosine: 0.16387629508972168
Step 500/2500, MSE: 0.1866629421710968, Cosine: 0.1636120080947876
Step 600/2500, MSE: 0.16461877524852753, Cosine: 0.14964210987091064
Step 700/2500, MSE: 0.18699654936790466, Cosine: 0.163763165473938
Step 800/2500, MSE: 0.18732553720474243, Cosine: 0.16396570205688477
Step 900/2500, MSE: 0.18750160932540894, Cosine: 0.16399991512298584
Step 1000/2500, MSE: 0.10415441542863846, Cosine: 0.10822218656539917
Step 1100/2500, MSE: 0.09545883536338806, Cosine: 0.10199916362762451
Step 1200/2500, MSE: 0.09560932219028473, Cosine: 0.10285413265228271
Step 1300/2500, MSE: 0.09461875259876251, Cosine: 0.10127240419387817
Step 1400/2500, MSE: 0.10546804964542389, Cosine: 0.10914766788482666
Step 1500/2500, MSE: 0.1066155731678

In [29]:
cos_sim = nn.CosineSimilarity(dim=1)
with torch.no_grad():
    final_output = startToEndLayerOutput(startLayerNum, targetLayerNum, getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding))
    final_output = model.pooler(final_output)

    # Compare with magic embedding
    sim_to_magic = cos_sim(final_output, magic_embedding) # should measure per token sim here

    print('Final results:')
    print(f'Similarity to target (pooled):')
    print(sim_to_magic.item())

Final results:
Similarity to target (pooled):
0.9009974598884583


In [30]:
import torch
import torch.nn.functional as F

# assumes EVERY token is at pos=1 which it might not be for bigger sequences
def process_all_embeddings(model, tokenPos=1):
    # Get the full embedding matrix
    word_embeddings = model.embeddings.word_embeddings.weight.data

    # Create position embedding for position 1 (or whatever position you want to use)
    position_embedding = model.embeddings.position_embeddings(
        torch.tensor([tokenPos], device=word_embeddings.device)
    )

    # Create token type embedding for type 0 (or whatever type you want to use)
    token_type_embedding = model.embeddings.token_type_embeddings(
        torch.tensor([0], device=word_embeddings.device)
    )

    # Process all tokens through the full embedding layer
    all_processed = []
    batch_size = 1024  # Process in batches to avoid memory issues

    for i in range(0, len(word_embeddings), batch_size):
        batch = word_embeddings[i:i + batch_size]
        # Add position and token type embeddings
        batch = batch + position_embedding + token_type_embedding
        # Apply layer norm and dropout (using eval mode so dropout is disabled)
        with torch.no_grad():
            batch = model.embeddings.LayerNorm(batch)
        all_processed.append(batch)

    return torch.cat(all_processed, dim=0)

def find_similar_tokens(processed_embeddings, target_embedding, tokenizer, top_k=10, filter_special=False):
    # Normalize embeddings for cosine similarity
    processed_embeddings = F.normalize(processed_embeddings, p=2, dim=1)
    target_embedding = F.normalize(target_embedding, p=2, dim=1)

    # Calculate cosine similarity
    # similarities = torch.matmul(target_embedding, processed_embeddings.T)
    similarities = F.cosine_similarity(target_embedding.unsqueeze(1), processed_embeddings.unsqueeze(0), dim=2)

    if filter_special:
        # Get all similarities and indices
        all_similarities, all_indices = torch.sort(similarities[0], descending=True)

        # Filter and collect top k meaningful tokens
        results = []
        for sim, idx in zip(all_similarities.squeeze(0), all_indices.squeeze(0)):
            if len(results) >= top_k:
                break

            token = tokenizer.convert_ids_to_tokens([idx.item()])[0]
            if (
                not token.startswith('[') and  # Skip special tokens
                not token.startswith('##') and  # Skip subword pieces
                not any(c in token for c in '〜་『』«»‰―⟩（') and  # Skip special characters
                # len(token) > 1 and  # Skip single characters
                token.isascii()  # Only keep ASCII tokens
            ):
                results.append((token, sim.item()))

        return results
    else:
        top_similarities, top_indices = torch.topk(similarities[0], k=top_k)
        # Original return for non-filtered case
        results = []
        for sim, idx in zip(top_similarities.squeeze(0), top_indices.squeeze(0)):
            token = tokenizer.convert_ids_to_tokens([idx.item()])[0]
            results.append((token, sim.item()))

        return results

In [31]:
highestSimilarityToken = []
for idx,i in enumerate(getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding)[0]):
    allVocabEmbeddings = process_all_embeddings(model, tokenPos=idx)
    target_embedding = i.unsqueeze(0)

    isMiddleToken = ((idx > 0) and ((idx+1) < len(getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding)[0])))
    similar_tokens = find_similar_tokens(allVocabEmbeddings, target_embedding, tokenizer, top_k=5, filter_special=isMiddleToken)

    if isMiddleToken:
        highestSimilarityToken.append(similar_tokens[0][0])

    print(similar_tokens)
    print('----------\n')

[('[CLS]', 1.0), ('[MASK]', 0.9065358638763428), ('[SEP]', 0.8594523668289185), ('[unused123]', 0.7592557668685913), ('[unused485]', 0.7588828206062317)]
----------

[('eki', 0.8003095984458923), ('kenji', 0.1835770457983017), ('akin', 0.17674842476844788), ('joshi', 0.17610540986061096), ('poznan', 0.17457808554172516)]
----------

[('670', 0.511936366558075), ('680', 0.22533312439918518), ('660', 0.21675491333007812), ('640', 0.1885528415441513), ('690', 0.18374300003051758)]
----------

[('laude', 0.8440538644790649), ('annum', 0.19670020043849945), ('moines', 0.18833157420158386), ('nedra', 0.18703509867191315), ('albans', 0.18675637245178223)]
----------

[('apostolic', 0.8747789859771729), ('papal', 0.26756876707077026), ('episcopal', 0.24843250215053558), ('dioceses', 0.23531778156757355), ('vatican', 0.23102545738220215)]
----------

[('tay', 0.7560009360313416), ('tai', 0.20952315628528595), ('dundee', 0.15035386383533478), ('tyne', 0.13950765132904053), ('sneered', 0.13883800

In [71]:
def slerp(embed1, embed2, t=0.5):
    # Ensure embeddings are normalized
    embed1 = F.normalize(embed1, p=2, dim=-1)
    embed2 = F.normalize(embed2, p=2, dim=-1)

    # Calculate dot product (cosine of angle between embeddings)
    dot = torch.sum(embed1 * embed2)

    # If vectors are very close, use linear interpolation
    if dot > 0.9995:
        return F.normalize(embed1 + t * (embed2 - embed1), p=2, dim=-1)

    # Clamp dot product to stay in acos's domain
    dot = torch.clamp(dot, -1.0, 1.0)

    # Calculate angle between vectors
    omega = torch.acos(dot)
    sin_omega = torch.sin(omega)

    # Calculate interpolation coefficients
    s1 = torch.sin((1.0 - t) * omega) / sin_omega
    s2 = torch.sin(t * omega) / sin_omega

    # Interpolate
    result = s1 * embed1 + s2 * embed2

    # Normalize result to ensure it's on the unit hypersphere
    return F.normalize(result, p=2, dim=-1)

In [74]:
with torch.no_grad():
    start_output = model(**tokenizer('apple', return_tensors='pt'), output_hidden_states=True)
    end_output = model(**tokenizer('weather', return_tensors='pt'), output_hidden_states=True)

# Calculate cosine similarity
cos_sim = nn.CosineSimilarity(dim=1)(start_output.pooler_output, end_output.pooler_output)
print(f"Cosine similarity: {cos_sim.item()}")

print('Similar tokens to start output')
for idx,i in enumerate(start_output.hidden_states[0][0]):
    allVocabEmbeddings = process_all_embeddings(model, tokenPos=idx)
    target_embedding = i.unsqueeze(0)

    isMiddleToken = ((idx > 0) and ((idx+1) < len(getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding)[0])))
    similar_tokens = find_similar_tokens(allVocabEmbeddings, target_embedding, tokenizer, top_k=5, filter_special=isMiddleToken)

    print(similar_tokens)
    print('----------\n')

print('\nSimilar tokens to end output')
for idx,i in enumerate(end_output.hidden_states[0][0]):
    allVocabEmbeddings = process_all_embeddings(model, tokenPos=idx)
    target_embedding = i.unsqueeze(0)

    isMiddleToken = ((idx > 0) and ((idx+1) < len(getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding)[0])))
    similar_tokens = find_similar_tokens(allVocabEmbeddings, target_embedding, tokenizer, top_k=5, filter_special=isMiddleToken)

    print(similar_tokens)
    print('----------\n')


midPointEmbedding = slerp(start_output.pooler_output, end_output.pooler_output)
print('\nSimilar tokens to middle of embeddings')
for idx,i in enumerate(midPointEmbedding):
    allVocabEmbeddings = process_all_embeddings(model, tokenPos=idx)
    target_embedding = i.unsqueeze(0)

    isMiddleToken = ((idx > 0) and ((idx+1) < len(getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding)[0])))
    similar_tokens = find_similar_tokens(allVocabEmbeddings, target_embedding, tokenizer, top_k=5, filter_special=isMiddleToken)

    print(similar_tokens)
    print('----------\n')

Cosine similarity: 0.9264100790023804
Similar tokens to start output
[('[CLS]', 1.0), ('[MASK]', 0.9065358638763428), ('[SEP]', 0.8594523668289185), ('[unused123]', 0.7592557668685913), ('[unused485]', 0.7588828206062317)]
----------

[('apple', 1.0000001192092896), ('apples', 0.5469692349433899), ('fruit', 0.2938264012336731), ('iphone', 0.267752468585968), ('itunes', 0.257937490940094)]
----------

[('[SEP]', 1.0000001192092896), ('[MASK]', 0.6763173341751099), ('[CLS]', 0.5776786804199219), ('.', 0.3778829872608185), ('the', 0.37075066566467285)]
----------


Similar tokens to end output
[('[CLS]', 1.0), ('[MASK]', 0.9065358638763428), ('[SEP]', 0.8594523668289185), ('[unused123]', 0.7592557668685913), ('[unused485]', 0.7588828206062317)]
----------

[('weather', 1.0), ('climate', 0.37233856320381165), ('meteorological', 0.36278918385505676), ('climates', 0.34757235646247864), ('weathered', 0.33171820640563965)]
----------

[('[SEP]', 1.0000001192092896), ('[MASK]', 0.67631733417510

In [42]:
def comparePooledOutputToEmbedding(words, embedding):
    with torch.no_grad():
        output = model(**tokenizer(words, return_tensors='pt'), output_hidden_states=True)
        pooled_output = output.pooler_output.squeeze()

        # Calculate cosine similarity
        cos_sim = torch.nn.functional.cosine_similarity(pooled_output, embedding)
        print(f"Cosine similarity: {cos_sim.item()}")

        return cos_sim.item()

def fullyComputeHiddenLayer(hiddenVals, cls, sep):
    layer_output = startToEndLayerOutput(startLayerNum, targetLayerNum, getHiddenLayerInput(startLayerNum, learned_hidden, cls_embedding, sep_embedding))
    return model.pooler(layer_output)

print(highestSimilarityToken)
comparePooledOutputToEmbedding(highestSimilarityToken, fullyComputeHiddenLayer(learned_hidden, cls_embedding, sep_embedding))

['magic']
Cosine similarity: 0.9999988079071045


0.9999988079071045

In [ ]:
def hook(module, input, output):
    print("Input:", input)  # input is a tuple
    print("Output:", output)
    return output  # return unchanged

modelPart = model.embeddings.word_embeddings

modelPart._forward_hooks.clear()
layernorm_hook_handle = model.embeddings.word_embeddings.register_forward_hook(hook)
layernorm_hook_handle = model.embeddings.position_embeddings.register_forward_hook(hook)
layernorm_hook_handle = model.embeddings.token_type_embeddings.register_forward_hook(hook)
layernorm_hook_handle = model.embeddings.LayerNorm.register_forward_hook(hook)
layernorm_hook_handle = model.embeddings.dropout.register_forward_hook(hook)
layernorm_hook_handle = model.embeddings.register_forward_hook(hook)
layernorm_hook_handle = model.encoder.layer[0].register_forward_hook(hook)

fantasy_output = model(**tokenizer('fantasy', return_tensors='pt'), output_hidden_states=True)

In [ ]:
modelPart._forward_hooks.clear()